In [1]:
import pandas as pd
import json

input_ = pd.read_excel('./3.item.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_ = input_[input_['Json_Compatibility'] != json.dumps('')].reset_index(drop=True)
input_['No'] = input_['No'].astype(int)

input_

,No,Item Number,Title,Subtitle,Price,Available,Sold,Vehicle W/o Engine,Vehicle W/ Engine,Picture,Url,Description Url,Json_Compatibility,Json_Src,Json_Specific
0,1,325960692572,Mazda CX-5 CX5 2012-2016 Electric Powerfold In...,,£148.95,5 available,7 sold,,,,https://www.ebay.co.uk/itm/325960692572?_ul=GB...,https://vi.vipr.ebaydesc.com/itmdesc/325960692...,"{""market"": ""UK"", ""page"": 5, ""data"": {""scopedCo...","{""0"": ""https://i.ebayimg.com/images/g/ZGcAAOSw...","{""0"": {""Condition"": ""New other (see details): ..."
1,2,325960692580,Mazda CX-5 CX5 2012-2016 Electric Powerfold In...,,£148.95,5 available,1 sold,,,,https://www.ebay.co.uk/itm/325960692580?_ul=GB...,https://vi.vipr.ebaydesc.com/itmdesc/325960692...,"{""market"": ""UK"", ""page"": 5, ""data"": {""scopedCo...","{""0"": ""https://i.ebayimg.com/images/g/EukAAOSw...","{""0"": {""Condition"": ""New other (see details): ..."
2,3,225616819852,Toyota Yaris 5dr 2020-2023 Electric Door Wing ...,,£129.95,,9 sold,,,,https://www.ebay.co.uk/itm/225616819852?_ul=GB...,https://vi.vipr.ebaydesc.com/itmdesc/225616819...,"{""market"": ""UK"", ""page"": 1, ""data"": {""scopedCo...","{""0"": ""https://i.ebayimg.com/images/g/38IAAOSw...","{""0"": {""Condition"": ""New other (see details): ..."
3,4,226014609106,Renault Clio MK5 2019-2024 Electric Powerfold ...,,£124.95,4 available,6 sold,,,,https://www.ebay.co.uk/itm/226014609106?_ul=GB...,https://vi.vipr.ebaydesc.com/itmdesc/226014609...,"{""market"": ""UK"", ""page"": 3, ""data"": {""scopedCo...","{""0"": ""https://i.ebayimg.com/images/g/UhYAAOSw...","{""0"": {""Condition"": ""New other (see details): ..."
4,5,226014609107,Renault Clio MK5 2019-2024 Electric Powerfold ...,,£124.95,5 available,,,,,https://www.ebay.co.uk/itm/226014609107?_ul=GB...,https://vi.vipr.ebaydesc.com/itmdesc/226014609...,"{""market"": ""UK"", ""page"": 3, ""data"": {""scopedCo...","{""0"": ""https://i.ebayimg.com/images/g/fiUAAOSw...","{""0"": {""Condition"": ""New other (see details): ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443,457,224030767402,For VW Crafter 2006-2017 Lower Door Wing Mirro...,,£7.95,5 available,33 sold,,,,https://www.ebay.co.uk/itm/224030767402?_ul=GB...,https://vi.vipr.ebaydesc.com/itmdesc/224030767...,"{""market"": ""UK"", ""page"": 10, ""data"": {""scopedC...","{""0"": ""https://i.ebayimg.com/images/g/NnIAAOSw...","{""0"": {""Condition"": ""New other (see details)""}..."
444,458,324184663257,For Mercedes Sprinter 2006-2019 Lower Wing Mir...,,£7.95,5 available,"1,002 sold",,,,https://www.ebay.co.uk/itm/324184663257?_ul=GB...,https://vi.vipr.ebaydesc.com/itmdesc/324184663...,"{""market"": ""UK"", ""page"": 115, ""data"": {""scoped...","{""0"": ""https://i.ebayimg.com/images/g/1~cAAOSw...","{""0"": {""Condition"": ""New other (see details)""}..."
445,459,325492805263,VW Fox 2003-2011 Heated Door Wing Mirror Glass...,,£7.95,5 available,46 sold,,,,https://www.ebay.co.uk/itm/325492805263?_ul=GB...,https://vi.vipr.ebaydesc.com/itmdesc/325492805...,"{""market"": ""UK"", ""page"": 5, ""data"": {""scopedCo...","{""0"": ""https://i.ebayimg.com/images/g/JT0AAOSw...","{""0"": {""Condition"": ""New other (see details): ..."
446,460,324184659149,For Ford Transit MK6/MK7 2000-2014 Door Wing M...,,£6.95,5 available,"5,779 sold",,,,https://www.ebay.co.uk/itm/324184659149?_ul=GB...,https://vi.vipr.ebaydesc.com/itmdesc/324184659...,"{""market"": ""UK"", ""page"": 28, ""data"": {""scopedC...","{""0"": ""https://i.ebayimg.com/images/g/X10AAOSw...","{""0"": {""Condition"": ""New other (see details)""}..."


In [2]:
from tqdm import tqdm

output = pd.DataFrame()
for i in tqdm(range(len(input_)), desc='Progress', ncols=77):
    json_ = json.loads(input_.loc[i, 'Json_Compatibility'])
    if json_['market'] == 'US':
        list_vehicle_url = [f'''https://www.ebay.com/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset={j*20}&module=COMPATIBILITY_TABLE''' for j in range(json_['page'])]
    elif json_['market'] == 'DE':
        list_vehicle_url = [f'''https://www.ebay.de/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset={j*20}&module=COMPATIBILITY_TABLE''' for j in range(json_['page'])]
    elif json_['market'] == 'UK':
        list_vehicle_url = [f'''https://www.ebay.co.uk/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset={j*20}&module=COMPATIBILITY_TABLE''' for j in range(json_['page'])]
    elif json_['market'] == 'AU':
        list_vehicle_url = [f'''https://www.ebay.com.au/g/api/finders?module_groups=PART_FINDER&referrer=VIEWITEM&offset={j*20}&module=COMPATIBILITY_TABLE''' for j in range(json_['page'])]
    df_temp = pd.DataFrame({'No': input_.loc[i, 'No'],
                            'Item Number': input_.loc[i, 'Item Number'],
                            'Market': json_['market'],
                            'Page': [j+1 for j in range(json_['page'])],
                            'JOIN_INP': [f'''{input_.loc[i, 'Item Number']};{j+1}''' for j in range(json_['page'])],
                            'Vehicle Url': list_vehicle_url,
                            'Data': json.dumps(json_['data'])})

    output = pd.concat([output, df_temp], ignore_index=True).fillna('')

output.to_excel('4.item_2.xlsx', index=False)

output

Progress: 100%|███████████████████████████| 448/448 [00:00<00:00, 811.78it/s]


,No,Item Number,Market,Page,JOIN_INP,Vehicle Url,Data
0,1,325960692572,UK,1,325960692572;1,https://www.ebay.co.uk/g/api/finders?module_gr...,"{""scopedContext"": {""catalogDetails"": {""categor..."
1,1,325960692572,UK,2,325960692572;2,https://www.ebay.co.uk/g/api/finders?module_gr...,"{""scopedContext"": {""catalogDetails"": {""categor..."
2,1,325960692572,UK,3,325960692572;3,https://www.ebay.co.uk/g/api/finders?module_gr...,"{""scopedContext"": {""catalogDetails"": {""categor..."
3,1,325960692572,UK,4,325960692572;4,https://www.ebay.co.uk/g/api/finders?module_gr...,"{""scopedContext"": {""catalogDetails"": {""categor..."
4,1,325960692572,UK,5,325960692572;5,https://www.ebay.co.uk/g/api/finders?module_gr...,"{""scopedContext"": {""catalogDetails"": {""categor..."
...,...,...,...,...,...,...,...
5948,461,224030762701,UK,30,224030762701;30,https://www.ebay.co.uk/g/api/finders?module_gr...,"{""scopedContext"": {""catalogDetails"": {""categor..."
5949,461,224030762701,UK,31,224030762701;31,https://www.ebay.co.uk/g/api/finders?module_gr...,"{""scopedContext"": {""catalogDetails"": {""categor..."
5950,461,224030762701,UK,32,224030762701;32,https://www.ebay.co.uk/g/api/finders?module_gr...,"{""scopedContext"": {""catalogDetails"": {""categor..."
5951,461,224030762701,UK,33,224030762701;33,https://www.ebay.co.uk/g/api/finders?module_gr...,"{""scopedContext"": {""catalogDetails"": {""categor..."
